In [1]:
path_model_to_eval = "experiments/attribute_prediction/2024-09-01_11-03-41/models/model_epoch_10.pth"  # TODO CHANGE

In [2]:
import os
import sys
import argparse

print(os.getcwd())
os.chdir("../../")
os.getcwd()

sys.argv = [
    "view",
    "--config",
    "config/single_task_attribute_prediction.yaml",
    "--model_path",
    path_model_to_eval,
]


parser = argparse.ArgumentParser()
parser.add_argument("--config", type=str, required=True, help="Path to the config file")
parser.add_argument(
    "--model_path", type=str, required=True, help="Path to the model pth"
)

args = parser.parse_args()

print(args.config)
print(args.model_path)

/data01/dl23vitcas/dl_project/src/single_task_attribute_prediction
config/single_task_attribute_prediction.yaml
experiments/attribute_prediction/2024-09-01_11-03-41/models/model_epoch_10.pth


In [3]:
import torch
from config_experiments import config, parse_args
from utils import set_seed, set_device
from dataloader import VOC08Attr
import torchvision.transforms as transforms
from model import AttributePredictionModel
from metrics import compute_mAP
import wandb

In [4]:
def mAP_view_attributes(mAP, data_test):
    print(mAP)
    print(f"\nmAP@0.50 (per class):")
    for i, value in enumerate(mAP):
        attribute = data_test.id2attribute[i + 1]
        print(f"AP {attribute}: {value:.2f}")
        wandb.config.update({f"AP {attribute} ": value})
    mAP = torch.mean(mAP)

    print(f"\nmAP : {mAP:.2f}")

    wandb.config.update({"mAP_attr@0.50": mAP})

In [5]:
model_path = parse_args().model_path
transform_test = transforms.Compose(
    [
        transforms.Resize(size=config["transform"]["resize_values"]),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=config["transform"]["mean"], std=config["transform"]["std"]
        ),
    ]
)
set_seed(config["global"]["seed"])
device = set_device(config["global"]["gpu_id"])


wandb.init(
    group="attribute prediction",
    project="DL",
    config=config,
    save_code=True,
    mode="disabled",
)

data_test = VOC08Attr(train=False, transform=transform_test)
model = AttributePredictionModel().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<All keys matched successfully>

In [6]:
mAP = compute_mAP(data_test, model, device)
mAP_view_attributes(mAP, data_test)

Compute mAP: 100%|██████████| 2227/2227 [03:30<00:00, 10.58it/s]


tensor([0.3495, 0.5960, 0.1510, 0.7580, 0.6092, 0.5733, 0.6805, 0.7405, 0.8661,
        0.7842, 0.8488, 0.8112, 0.7947, 0.7743, 0.8074, 0.8038, 0.7954, 0.7254,
        0.7974, 0.8220, 0.7146, 0.8006, 0.2078, 0.6808, 0.8141, 0.6867, 0.8093,
        0.6553, 0.5226, 0.4353, 0.5990, 0.2827, 0.7758, 0.8016, 0.5856, 0.6182,
        0.6844, 0.5850, 0.8121, 0.6354, 0.6925, 0.6399, 0.5136, 0.1927, 0.5533,
        0.5062, 0.8338, 0.3841, 0.5673, 0.8203, 0.8668, 0.8077, 0.8484, 0.4790,
        0.6108, 0.8510, 0.8767, 0.7066, 0.8052, 0.6275, 0.3485, 0.5000, 0.8333,
        0.1306], device='cuda:0')

mAP@0.50 (per class):
AP 2D Boxy: 0.35
AP 3D Boxy: 0.60
AP Round: 0.15
AP Vert Cyl: 0.76
AP Horiz Cyl: 0.61
AP Occluded: 0.57
AP Tail: 0.68
AP Beak: 0.74
AP Head: 0.87
AP Ear: 0.78
AP Snout: 0.85
AP Nose: 0.81
AP Mouth: 0.79
AP Hair: 0.77
AP Face: 0.81
AP Eye: 0.80
AP Torso: 0.80
AP Hand: 0.73
AP Arm: 0.80
AP Leg: 0.82
AP Foot/Shoe: 0.71
AP Wing: 0.80
AP Propeller: 0.21
AP Jet engine: 0.68
AP Window: 0